# Daily Notebook #2

### Current State

- Current Course: Data Analyst in Python: Data Manipulation
- Todays Notes: datacamp course chapter reviewed groupbys and pivot tables.

### Goals

- I had a thought - to start off with notebooks before I start the study process to review what I learned yesterday. That way im implementing some repetition and recall. So every night after study, ill write down the main concepts learned, and the next day as a start to the study session I will code my daily notebook

### Dataset for Today

Choclate sales - https://www.kaggle.com/datasets/atharvasoundankar/chocolate-sales

### Code

In [1]:
import pandas as pd

df = pd.read_csv('Chocolate Sales.csv')
df.head()

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04-Jan-22,"$5,320",180
1,Van Tuxwell,India,85% Dark Bars,01-Aug-22,"$7,896",94
2,Gigi Bohling,India,Peanut Butter Cubes,07-Jul-22,"$4,501",91
3,Jan Morforth,Australia,Peanut Butter Cubes,27-Apr-22,"$12,726",342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24-Feb-22,"$13,685",184


### Data Cleaning

Carrying over some fixes that I figured out in my previous notebook. Added a resource to get the datetime formatting correct. 

In [2]:
# Data cleaning

#I already knew some of the issues with the dataset and fix them here. 
#https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
df['Amount'] = df['Amount'].str.replace(r'[$,]', '', regex=True).astype(int)
df['Date'] = pd.to_datetime(df['Date'], format = '%d-%b-%y')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Sales Person   1094 non-null   object        
 1   Country        1094 non-null   object        
 2   Product        1094 non-null   object        
 3   Date           1094 non-null   datetime64[ns]
 4   Amount         1094 non-null   int32         
 5   Boxes Shipped  1094 non-null   int64         
dtypes: datetime64[ns](1), int32(1), int64(1), object(3)
memory usage: 47.1+ KB


### Analysis

Goal: 2 groupbys and 2 pivot tables

First starting with the basic creations to practice the syntax:

In [4]:
#Total Sales By Sales person
top_sales = df.groupby('Sales Person')['Amount'].sum().reset_index().sort_values('Amount', ascending = False)
top_sales.head(10)

,Sales Person,Amount
5,Ches Bonnell,320901
20,Oby Sorrel,316645
17,Madelene Upcott,316099
3,Brien Boise,312816
16,Kelci Walkden,311710
23,Van Tuxwell,303149
7,Dennison Crosswaite,291669
2,Beverie Moffet,278922
14,Kaine Padly,266490
19,Marney O'Breen,259742


In [5]:
#Total Sales by Sales person Pivot Table
total_sales = df.pivot_table(values ='Amount', index = 'Sales Person', aggfunc = 'sum') \
                .reset_index() \
                .sort_values('Amount', ascending = False)
total_sales.head(10)

,Sales Person,Amount
5,Ches Bonnell,320901
20,Oby Sorrel,316645
17,Madelene Upcott,316099
3,Brien Boise,312816
16,Kelci Walkden,311710
23,Van Tuxwell,303149
7,Dennison Crosswaite,291669
2,Beverie Moffet,278922
14,Kaine Padly,266490
19,Marney O'Breen,259742


In [6]:
#best performing products per country. Goal is to practice multiple varaibles grouped. Also want to show total margins
prod_per_country = df.groupby(['Country', 'Product'])['Amount'].sum().reset_index()
prod_per_country

,Country,Product,Amount
0,Australia,50% Dark Bites,89222
1,Australia,70% Dark Bites,39354
2,Australia,85% Dark Bars,38479
3,Australia,99% Dark & Pure,54908
4,Australia,After Nines,27769
...,...,...,...
127,USA,Peanut Butter Cubes,33628
128,USA,Raspberry Choco,83524
129,USA,Smooth Sliky Salty,45493
130,USA,Spicy Special Slims,25214


In [7]:
#I want to try pulling out the top product per country, since the last dataframe is informative but hard to digest
#I cant think of a way to do this off the top, think I need to loop through countries and then get the max from prev df
#prod_per_country[(prod_per_country['Country'] == 'Australia')].sort_values(by = 'Amount', ascending = False)
countries = prod_per_country['Country'].unique()

all_data = []
for country in countries:
    df = prod_per_country[(prod_per_country['Country'] == country)].sort_values(by = 'Amount', ascending = False).head(1)
    all_data.append(df)

top_products_per_country = pd.concat(all_data)

In [8]:
top_products_per_country.sort_values('Amount', ascending = False)

,Country,Product,Amount
0,Australia,50% Dark Bites,89222
80,New Zealand,Mint Chip Choco,86709
128,USA,Raspberry Choco,83524
105,UK,Peanut Butter Cubes,79695
54,India,Eclairs,79009
41,Canada,Smooth Sliky Salty,68257


Notes on the above code - I had to fumble around a little bit, and ultimately got the idea to concat outside the for loop from google - was a recommended performance improver. I thought that was good to note and practice - create the dataframes, save to a list the concat at once. 

I struggled for a while on the final formatting and concating of teh dataframes - formatting was strange. Was using .iloc[0] to ge tthe top value per country and didnbt work well. finally remembered to use head. Going to look into a better solution or see how close chatgpt would get to what I did.

AI Solution:

In [9]:
prod_per_country.loc[prod_per_country.groupby(['Country'])['Amount'].idxmax()].reset_index(drop = True)

,Country,Product,Amount
0,Australia,50% Dark Bites,89222
1,Canada,Smooth Sliky Salty,68257
2,India,Eclairs,79009
3,New Zealand,Mint Chip Choco,86709
4,UK,Peanut Butter Cubes,79695
5,USA,Raspberry Choco,83524


My issues:
- I didnt know about idxmax function
- still feeling fuzzy after reading explanations about why we are using loc -> but I think I understand that it is the index label, and with index labels we use loc. 
- Also noted from research that you should reset_index, drop = true, inplace = true if you want iloc consistency. So index 1 is positon 1. 

# Reflection

 - Learned about inxmax()
 - learned a little more about loc and iloc 
 - practiced the basics of grouping and pivots. Didnt get as advanced as I wanted but I fell down a rabbit hole trying to challenge myself to do something I didnt really know how to do right away - so thats fine
 